# 准备

In [1]:

from einops import rearrange, reduce
import numpy as np
from utils import guess

In [3]:
x = np.random.RandomState(42).normal(size=[10, 32, 100, 200])

# 选择深度学习框架

此处选用 PyTorch。

In [4]:
import torch
x = torch.from_numpy(x)
x.requires_grad = True

In [5]:
type(x), x.shape

(torch.Tensor, torch.Size([10, 32, 100, 200]))

# 简单计算

在 CV 中，在 bchw 和 bwhc 之间互相转换是一个常用的操作。

In [6]:
y = rearrange(x, 'b c h w -> b h w c')
guess(y.shape)

Answer is: (10, 100, 200, 32) (hover to see)

# 反向传播

- 梯度是深度学习的基石之一。
- 像对待框架原生的操作一样，`einops` 操作支持反向传播。

In [7]:
y0 = x
y1 = reduce(y0, 'b c h w -> b c', 'max')
y2 = rearrange(y1, 'b c -> c b')
y3 = reduce(y2, 'c b -> ', 'sum')

y3.backward()
print(reduce(x.grad, 'b c h w -> ', 'sum'))

tensor(320., dtype=torch.float64)


# `einops.asnumpy`

框架的张量将被转换成数组。如果张量在 GPU 上，那么它被自动拉回 CPU。

In [8]:
from einops import asnumpy
y3_numpy = asnumpy(y3)

print(type(y3_numpy))

<class 'numpy.ndarray'>


# 使用 `einops` 完成深度学习中的常见操作

In [9]:
x.shape

torch.Size([10, 32, 100, 200])

In [10]:
# 展平（flattening）常见于两个卷积层或全连接层之间
y = rearrange(x, 'b c h w -> b (c h w)')
guess(y.shape)

Answer is: (10, 640000) (hover to see)

In [11]:
# 空间维度转深度维度
y = rearrange(x, 'b c (h h1) (w w1) -> b (h1 w1 c) h w', h1=2, w1=2)
guess(y.shape)

Answer is: (10, 128, 50, 100) (hover to see)

In [12]:
# 上面的逆操作
y = rearrange(x, 'b (h1 w1 c) h w -> b c (h h1) (w w1)', h1=2, w1=2)
guess(y.shape)

Answer is: (10, 8, 200, 400) (hover to see)

# 缩减

In [14]:
x.shape

torch.Size([10, 32, 100, 200])

In [13]:
# 全局平均池化
y = reduce(x, 'b c h w -> b c', reduction='mean')
guess(y.shape)

Answer is: (10, 32) (hover to see)

In [15]:
# 2x2 最大池化
y = reduce(x, 'b c (h h1) (w w1) -> b c h w', reduction='max', h1=2, w1=2)
# 等价于：
# y = reduce(x, 'b c (h 2) (w 2) -> b c h w', reduction='max')
guess(y.shape)

Answer is: (10, 32, 50, 100) (hover to see)

1d, 2d, 3d 池化的定义方式十分相似：

```python
reduce(x, '(t 2) b c -> t b c', reduction='max')    # 1d
reduce(x, 'b c (x 2) (y 2) (z 2) -> b c x y z', reduction='max')    # 3d
```

这种一致性是 einops 的一大优势。

# squeeze 和 unsqueeze

In [19]:

# models typically work only with batches, 
# so to predict a single image ...
image = rearrange(x[0, :3], 'c h w -> h w c')
# ... create a dummy 1-element axis ...
y = rearrange(image, 'h w c -> () c h w')
# ... imagine you predicted this with a convolutional network for classification,
# we'll just flatten axes ...
predictions = rearrange(y, 'b c h w -> b (c h w)')
# ... finally, decompose (remove) dummy axis
predictions = rearrange(predictions, '() classes -> classes')
predictions.shape

torch.Size([60000])

# keepdims-like behavior for reductions

In [23]:
# 空括号 () 提供了一个长为 1 的维度，这维度可以广播。
y = x - reduce(x, 'b c h w -> b c () ()', 'mean')
# 也可以使用 1 来引入一个新维度，效果和 () 一样。
z = x - reduce(x, 'b c h w -> b c 1 1', 'mean')
print(torch.equal(y, z))
guess(z.shape)

True


Answer is: (10, 32, 100, 200) (hover to see)

In [24]:
# 针对整个 batch 的逐 channel 均值归一化：
y = x - reduce(y, 'b c h w -> 1 c 1 1', 'mean')
guess(y.shape)

Answer is: (10, 32, 100, 200) (hover to see)

# Stack 堆叠

本节内容与基础教程对应小节重复，故略。

# 在单个 channel 中打乱

称为 channel shuffle，该操作在 ShuffleNet 的论文中出现。

In [31]:
y = rearrange(x, 'b (g1 g2 c) h w-> b (g2 g1 c) h w', g1=4, g2=4)
guess(y.shape)

Answer is: (10, 32, 100, 200) (hover to see)

In [32]:
# 简化的 channel shuffle：
y = rearrange(x, 'b (g c) h w-> b (c g) h w', g=4)
guess(y.shape)

Answer is: (10, 32, 100, 200) (hover to see)

# layer API

许多框架支持以层的方式直接指定网络结构。为此 `einops` 提供了一种层写法。

导入对应 PyTorch 的 API：

```python
from einops.layers.torch import Rearrange, Reduce
```

layer API 的写法和标准 API 基本一致，唯一的不同是省去了第一个参数（要处理的张量）。张量不再显式给出，而是在层的传递过程中自动得到。

```python
layer = Rearrange(pattern, **axes_lengths)
layer = Reduce(pattern, reduction, **axes_lengths)

# 对张量应用 layer
x = layer(x)
```

下面的代码展示了用 layer API 改进用原始 PyTorch 代码编写的简单卷积网络的过程。

原始代码：
```python
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

conv_net_old = Net()
```

改进后的版本：
```python
conv_net_new = nn.Sequential(
    nn.Conv2d(1, 10, kernel_size=5),
    nn.MaxPool2d(kernel_size=2),
    nn.ReLU(),
    nn.Conv2d(10, 20, kernel_size=5),
    nn.MaxPool2d(kernel_size=2),
    nn.ReLU(),
    nn.Dropout2d(),
    Rearrange('b c h w -> b (c h w)'),
    nn.Linear(320, 50),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(50, 10),
    nn.LogSoftmax(dim=1)
)
```